In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle,time
from sklearn import datasets, linear_model,KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer,confusion_matrix,mean_absolute_error
from sklearn.svm import LinearSVC

x = pd.read_csv(r'C:\Users\MUR2HO\Data\Features\Top15_Merged_Rank1_Features_Spindle_W2_and_W3_all.csv', encoding = 'ISO-8859-1', sep=';')
y = pd.read_csv(r'C:\Users\MUR2HO\Data\HPC\Qs Daten_04.10_18_19.02_11.06_Durchmesser und Konzentrizitaet_paarweise_vertauscht.csv', encoding = 'ISO-8859-1', sep=';')

x = x.T[1:30].T   ###All the features

y1 = y.T[2:3].T
y=y1.to_numpy()   ###convert dataframe to array
X1 = x
X=X1.to_numpy()  # X is the feature set and y is the target


def pretty_table(rows, column_count, column_spacing=2):
    aligned_columns = []
    for column in range(column_count):
        column_data = list(map(lambda row: row[column], rows))
        aligned_columns.append((max(map(len, column_data)) + column_spacing, column_data))

    for row in range(len(rows)):
        aligned_row = map(lambda x: (x[0], x[1][row]), aligned_columns)
        yield ''.join(map(lambda x: x[1] + ' ' * (x[0] - len(x[1])), aligned_row))
        
rows = [['Iteration', 'R^2_train', 'R^2_test', 'MAE', 'MAX', 'MIN', 'MAPE', 'Training_Time', 'Testing_Time'  ]]        


print('List was read')


# Repeated K-fold cross validation

n = 10  # repeat the CV procedure 10 times to get more precise results
    
for i in range(n):
    kfold = KFold(n_splits=10, shuffle=True, random_state=i) #random_state is used to get different splits of data in each run
    
######## K-fold cross validation

    for train_ix, test_ix in kfold.split(X):
        
# Different splits for X_train, X_test, y_train and y_test in each run

        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        X_train = pd.DataFrame(train_X)
        X_test = pd.DataFrame(test_X)
        y_train = pd.DataFrame(train_y)
        y_test = pd.DataFrame(test_y)
        #print(X_test)
        
    from sklearn.ensemble import RandomForestRegressor
    rfr = RandomForestRegressor(n_estimators=100, max_features='sqrt', criterion='mae', min_samples_split=3)
    
    from sklearn.ensemble import ExtraTreesRegressor
    #rfr = ExtraTreesRegressor(n_estimators=300, max_features='auto', criterion='mae', min_samples_split=3)

    from sklearn.ensemble import AdaBoostRegressor
    #rfr = AdaBoostRegressor(n_estimators=100, loss='exponential')

    
    from sklearn.ensemble import GradientBoostingRegressor
    #rfr = GradientBoostingRegressor(loss='lad', max_depth=5, min_samples_split = 4, n_estimators=3000)
    
    from sklearn import tree
    #rfr = tree.DecisionTreeRegressor(max_depth=5)
    
    
    from sklearn.linear_model import LinearRegression
    #rfr = LinearRegression() 
    
    
    from sklearn import linear_model
    #rfr=linear_model.BayesianRidge()
    
    
    from sklearn.svm import SVR
    #rfr = SVR()

    
    from sklearn.neural_network import MLPRegressor

    #rfr= mlp= MLPRegressor(hidden_layer_sizes=(8,100, 100, 1), activation ='relu', learning_rate_init=0.01)
    
        
    start = time.time()
    rfr.fit(X_train, y_train)
    stop = time.time()

    start2 = time.time()
    predictions = rfr.predict(X_test)
    stop2 = time.time()
        
# calculate the parameters for each iteration
    
    d=y_test.T
    
    ### Delta
    delta= abs(predictions-d)
    delta

    ### R^2
    R_2 = rfr.score(X_train, y_train)
    R_2 = round(R_2, 6)
    R_quadrat = rfr.score(X_test, y_test)
    R_quadrat = round(R_quadrat, 6)
 
    ###MAE
   
    MAE=mean_absolute_error(y_test, predictions)
    MAE = round(MAE, 6)

     ### MAX
  
    b=delta.T
    maxi=np.max(b).values
 
    ### MIN
   
    mini=np.min(b).values
   

    ### MAPE
    def my_compute(original, predict):
        return abs((original-predict)/original)
    z=my_compute(d, predictions)
    a=z.T
    mape=(a.sum(axis=0)*100/len(a)).values
    # print('MAPE:   ', mape)
    
    #Time 
    training_time = round(stop-start, 6)
    testing_time = round(stop2-start2, 6)

      
# loop for storing the values
    
       
    rows.append((str(i), str(R_2), str(R_quadrat), str(MAE), str(maxi), str(mini), str(mape), str(training_time), str(testing_time) ))

    
    for line in pretty_table(rows, 9):
        print(line)